# CLEAN DATA
Remove the galactic dust and the galactic dipole.

In [ ]:
import healpy as hp
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import statistics as sts
from scipy.linalg import lstsq